In [106]:
import numpy as np 
import random
import pprint
import operator
import copy
import time
from itertools import combinations
from mainCliqueOptimo import cliqueMaximo

In [107]:
#CARGAR MATRIZ INCIDENCIA RANDOM, CONTROLO EL % DE CONEXIONES(1)
def matrizIncidencia(cantNodos:int, probabilidad:float):
    matriz=np.zeros((cantNodos,cantNodos),int)
    for fila in range(cantNodos):
        for columna in range(cantNodos):
            if fila==columna:
                matriz[fila][columna]=0
            elif random.random()<probabilidad:
                #CONTROLO CANT DE 1
                matriz[fila][columna]=1
    return matriz

In [108]:
#DE MATRIZ DE INCIDENCIA HAGO LISTA DE CONEXIONES
def listaConexiones(matriz:np.ndarray):
    lista=[]
    cantNodos=matriz.shape[0]
    for fila in range(cantNodos):
        columna=-1
        conexion=False
        for elemento in matriz[fila]:
            #ME EMPIEZO A MOVER X LAS COLUMNAS
            columna+=1
            if elemento==1:
                lista.append((fila,columna))
                conexion=True
        if conexion==False:
            #NODO SIN ARISTAS
            lista.append((fila,None))
    return lista

In [109]:
class TDAGrafo:
    def __init__(self,listaConexiones):
        self.grafo=dict()
        self.listaADiccionario(listaConexiones)
        self.cardinalesDicc=dict()
        #self.cardinalidades()


    #DE LISTA DE CONEXIONES A DICCIONARIO
    def listaADiccionario(self,listaConexiones):
    
        for element1,element2 in listaConexiones:
            if element2 !=None:

                self.grafo.setdefault(element1,[]) #ESTRUCTURA
                if element2 not in self.grafo[element1]: #NO DUPLICADOS
                    self.grafo[element1].append(element2)

                self.grafo.setdefault(element2,[]) #ESTRUCTURA
                if element1 not in self.grafo[element2]: #NO DUPLICADOS
                    self.grafo[element2].append(element1)

            else:
                self.grafo.setdefault(element1,[])


    #SEGÚN UN CLIQUE, DECIR SI UN NODO ES VECINO A TODOS SUS ELEMENTOS
    def esVecino(self,clique:list,nodo:int):
        for elemento in clique:
            #ME FIJO EN EL DICCIONARIO DEL NODO SI LOS ELEMENTOS DEL CLIQUE ESTÁN EN EL
            #CON QUE 1 ELEMENTO NO ESTÉ, YA EL NODO NO ES VECINO DEL CLIQUE
            if elemento not in self.grafo[nodo]:
                return False
        return True


    #HACE TODAS LAS COMBINACIONES DE LOS VERTICES DE UNA CARDINALIDAD Y DICE SI SON CLIQUE(Y LO DEVUELVE)
    def combinacionEsClique(self,vertices, cardiClique):
        combinaciones=list(combinations(vertices, cardiClique))
        print('COMBINACIONES:{}, {}'.format(combinaciones,cardiClique)) 
        for combinacion in combinaciones:
            if self.esClique(combinacion):
                return combinacion
        return False
        

    #DADA UNA COMBINACION SE FIJA SI ENTRE ELLOS SON VECINOS
    def esClique(self,combinacion):
        for nodo in combinacion:
            for elemento in combinacion:
                if nodo != elemento:
                    if nodo not in self.grafo[elemento]:
                        return False
        return True





In [110]:
def maxCliqueMona(grafo):
    inicio=time.clock()
    pila=pilaCardinales(copiarGrafo(grafo.grafo))
    
    clique=[]
    
    while not pila.estaVacia():
        nodoTratar=pila.desapilar()
        if len(clique)==0:
            clique.append(nodoTratar)
        elif grafo.esVecino(clique,nodoTratar):
            clique.append(nodoTratar)
        else:
            final=time.clock()-inicio
            return clique,final
    
    final=time.clock()-inicio
    return clique,final

        



In [111]:
def nodoYcardinalDic(grafo):
    #DICCIONARIO DE CARDINALES ORIGINAL
    cardinalidades=dict()
    for nodo,conectados in grafo.items():
        cardinalidades[nodo]=len(conectados) #NO TIENE ORDEN
        #print(cardinalidades[nodo])
    
    return cardinalidades

In [112]:
#DADO UN NODO LO ELIMINA DEL DICCIONARIO DE CARDINALIDADES X NODO
def eliminarNodo(grafoAux,nodo):
    conexionesNodo=grafoAux[nodo]
    for conexion in conexionesNodo:
        grafoAux[conexion].remove(nodo)
    grafoAux.pop(nodo)
    return grafoAux
    


In [113]:
#DEVUELVE PILA CON CARDINALIDADES
def pilaCardinales(grafoAux):
    pilaFinal=TDAPila()
    listaOrdenada=sorted(nodoYcardinalDic(grafoAux).items(), key=lambda x: x[1])

    while len(listaOrdenada)>0:
        pilaFinal.apilar(listaOrdenada[0][0])
        grafoAux=eliminarNodo(grafoAux,listaOrdenada[0][0])
        
        listaOrdenada=sorted(nodoYcardinalDic(grafoAux).items(), key=lambda x: x[1])
    return pilaFinal

In [114]:
def copiarGrafo(grafo):
    grafoAux=copy.deepcopy(grafo)
    return grafoAux

In [115]:
class TDAPila:
     def __init__(self):
         self.items = []

     def estaVacia(self):
         return self.items == []

     def apilar(self, item):
         self.items.append(item)

     def desapilar(self):
        """ Devuelve el elemento tope y lo elimina de la pila.
        Si la pila está vacía levanta una excepción. """
        try:
            return self.items.pop()
        except IndexError:
            raise ValueError("La pila está vacía")

     def inspeccionar(self):
         return self.items[len(self.items)-1]

     def tamano(self):
         return len(self.items)

In [116]:
#DADO UN VALOR, DEVUELVE UN ARRAY CON TODOS LOS NODOS DE ESA CARDINALIDAD PARA ARRIBA INCLUSIVE
def nodosSuperiores(grafo, valor):
    lista=list(grafo.cardinalesDicc.keys())
    lista.sort()
    print('DICCIONARIO CARDINALES LISTA:{} '.format(lista))
    arrayNodosSup=[]
    arrayCardinalesSup=[]
    for cardi in lista:
        if cardi>=valor:
            arrayCardinalesSup.append(cardi)
    
    print('ARRAY CARDINALES SUP:{} '.format(arrayCardinalesSup))
    
    for card in arrayCardinalesSup:
         for nodo in grafo.cardinalesDicc[card]:
                arrayNodosSup.append(nodo)
    print('Nodos Superiores para cardinalidad {}: {}'.format(valor,arrayNodosSup))
    return arrayNodosSup

In [118]:
cantNodos=9
probabilidad=0.3



m1 = np.array([
    [0,1,1,1,0,1,1,0],
    [1,0,1,1,1,1,0,0],
    [1,1,0,1,0,0,1,1],
    [1,1,1,0,1,0,0,1],
    [0,1,0,1,0,0,0,0],
    [1,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0],
    [0,0,1,1,0,0,0,0]])

m=matrizIncidencia(cantNodos,probabilidad)

m2= np.array([
    [0,1,1,1,1],
    [1,0,1,1,1],
    [1,1,0,1,1],
    [1,1,1,0,1],
    [1,1,1,1,0]
])

m3= np.array([
    [0,1,0,0,0],
    [1,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])

print(m)


l=listaConexiones(m)
#print(l)
print()

grafo=TDAGrafo(l)
print()
pretty_print = pprint.PrettyPrinter()
pretty_print.pprint(grafo.grafo)




resultado,tiempo=maxCliqueMona(grafo)
resultado1,tiempo1=cliqueMaximo(grafo, cantNodos)
print()
print('heuris = {}, tiempo={} milisegundos'.format({x for x in resultado},tiempo))
print('optimo = {}, tiempo={} milisegundos'.format({x for x in resultado1},tiempo1))


[[0 1 0 1 0 0 1 0 0]
 [0 0 1 0 0 1 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [0 0 1 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [1 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]]


{0: [1, 3, 6, 7],
 1: [0, 2, 5, 6, 8],
 2: [1, 4, 5, 3],
 3: [0, 2, 4, 6, 8, 7],
 4: [2, 3],
 5: [1, 2],
 6: [0, 1, 3, 8],
 7: [0, 3],
 8: [3, 6, 1]}

heuris = {8, 3, 6}, tiempo=8.280000000127075e-05 milisegundos
optimo = {8, 3, 6}, tiempo=0.002157199999999193 milisegundos


In [119]:
calvesordenadas = sorted({1:1,89:7,22:9,4:34,3:90})
print(calvesordenadas[-1])
calvesordenadas

89


[1, 3, 4, 22, 89]

In [120]:
valor=2
listita=[0,1,2,3,4,5]
print(listita[valor-1:])

[1, 2, 3, 4, 5]
